In [16]:
import datetime
import pytz
import time



date = "2017/8/16 13:29:43"
#utc = pytz.timezone('Etc/UTC')
def dateToUnixConverter(date):
    #est = pytz.timezone('America/New_York')
    date_convert = [date.split(" ")[0].split("/"), date.split(" ")[1].split(":")]
    dateDT = datetime.datetime(int(date_convert[0][0]), int(date_convert[0][1]), int(date_convert[0][2]), \
                           int(date_convert[1][0])-3, int(date_convert[1][1]), int(date_convert[1][2]))#,\
                          #tzinfo = est)
    return int(time.mktime(dateDT.timetuple()))*1000000000

#utc_dt = dateDT.astimezone(utc)
#print("datetime object", dateDT)
#epoch = datetime.datetime.utcfromtimestamp(0)
#unixtime = (utc_dt - epoch).total_seconds() 
#unixtime = time.mktime(utc_dt.timetuple())


    

#unixtime 1502904583.0
dateToUnixConverter(date)


1502904583000000000

In [17]:
#Query Data - works and has been QCed on  live instance

import argparse

from influxdb import InfluxDBClient

host='localhost'
port=8086
user = 'root'
password = 'root'
dbname = 'mydb'
query = 'SELECT * from stockprice;'
client = InfluxDBClient(host, port, user, password, dbname)

print("Queying data: " + query)
result = client.query(query)

print("Result: {0}".format(result))

ImportError: No module named 'influxdb'

In [ ]:
#script to add data


import argparse
from influxdb import InfluxDBClient
import datetime
import pytz
import time


date = "2017/8/16 13:29:43"
#utc = pytz.timezone('Etc/UTC')
def dateToUnixConverter(date):
    #est = pytz.timezone('America/New_York')
    date_convert = [date.split(" ")[0].split("/"), date.split(" ")[1].split(":")]
    dateDT = datetime.datetime(int(date_convert[0][0]), int(date_convert[0][1]), int(date_convert[0][2]), \
                           int(date_convert[1][0])-3, int(date_convert[1][1]), int(date_convert[1][2]))#,\
                          #tzinfo = est)
    return time.mktime(dateDT.timetuple())*1000000000


host='localhost'
port=8086
user = 'root'
password = 'root'
dbname = 'mydb'
client = InfluxDBClient(host, port, user, password, dbname)
json_body = [
        {
            "measurement": "stockprice2",
            "tags": {
                "ticker": "aapl"
            },
            "time": dateToUnixConverter(date),
            "fields": {
                "price": 123,
                "Var": 2}
        }
    ]


print("Write points: {0}".format(json_body))
client.write_points(json_body)




In [ ]:
import pytz
pytz.all_timezones

In [ ]:
import pandas as pd
import numpy as np
import requests, os
import time
from matplotlib import pyplot
%matplotlib inline  

lags_to_forecast = 10
print("done")